<div style="text-align: center; background-color: #0A6EBD; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Kaggle Competitions: Learning Agency Lab - Automated Essay Scoring 2.0  @FIT-HCMUS, VNU-HCM 📌
</div>

<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 02 - BUILD MODEL 📌
</div>

# Import các thư viện cần thiết


In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import re


import warnings
warnings.filterwarnings('ignore')

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv
/kaggle/input/aes-hcmus-whl/pyproject_toml-0.0.10-py3-none-any.whl
/kaggle/input/aes-hcmus-whl/textstat-0.7.3-py3-none-any.whl
/kaggle/input/aes-hcmus-whl/pyspellchecker-0.8.1-py3-none-any.whl
/kaggle/input/aes-hcmus-whl/pyphen-0.15.0-py3-none-any.whl
/kaggle/input/data-processed/train_processed.csv
/kaggle/input/data-processed/test_processed.csv


In [6]:
# Use for pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import cohen_kappa_score

# Use for training model
from optuna.samplers import TPESampler
from nltk.tokenize import word_tokenize
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier 
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer
import lightgbm as lgb

In [7]:
df_train = pd.read_csv('/kaggle/input/data-processed/train_processed.csv')
df_test = pd.read_csv('/kaggle/input/data-processed/test_processed.csv')


### Cài đặt Pipeline

In [8]:
df_train

,score,word_count,essay_length,sentences_count,paragraph_count,clean_text,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,misspelled_count
0,3,498,2677,13,1,many people have car where they live. the thin...,56.97,15.1,11.4,7.21,18.0,8.40,57.0,13.000000,26
1,3,332,1669,19,9,i am a scientist at nasa that is discussing th...,87.35,5.5,6.8,4.52,5.9,6.11,22.0,6.714286,8
2,4,550,3077,24,7,people always wish they had the same technolog...,65.05,9.9,11.4,8.36,11.1,6.92,59.0,15.500000,9
3,4,451,2701,23,9,"we all heard about venus, the planet without a...",58.11,10.5,13.0,10.15,12.2,7.95,71.0,15.500000,10
4,3,373,2208,15,11,"dear, state senator this is a letter to argue ...",54.46,11.9,12.7,9.99,13.5,7.52,51.0,19.333333,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17302,2,157,856,9,5,"the story ""the challenge of exploing venus ""is...",70.94,7.6,10.0,7.60,8.0,7.44,21.0,14.750000,13
17303,4,579,3354,28,11,technology ha changed a lot of way that we liv...,56.39,11.2,11.7,9.46,12.2,7.21,65.0,14.500000,19
17304,2,215,1122,15,5,if you do not like sitting around all day than...,90.39,4.3,6.4,5.33,5.1,1.52,9.0,6.571429,2
17305,1,231,1427,11,1,"in ""the challenge of exporing venus,"" the auth...",58.42,10.4,11.4,11.20,12.9,8.69,46.0,14.000000,10


In [9]:
vectorizer = TfidfVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(3,6),
            min_df=0.05,
            max_df=0.95,
            sublinear_tf=True,
)

train_tfid = vectorizer.fit_transform([i for i in df_train['clean_text']])
dense_matrix = train_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns

## Dataframe sinh ra sau khi vector tfidf hóa cột clean_text

In [10]:
df

,tfid_0,tfid_1,tfid_2,tfid_3,tfid_4,tfid_5,tfid_6,tfid_7,tfid_8,tfid_9,...,tfid_18901,tfid_18902,tfid_18903,tfid_18904,tfid_18905,tfid_18906,tfid_18907,tfid_18908,tfid_18909,tfid_18910
0,0.013104,0.000000,0.027252,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.014123,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.049004,0.000000,0.0,0.0,0.000000,...,0.014383,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.031132,0.000000,0.031020,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
3,0.012223,0.014909,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.028272,0.015672,0.015793,0.0,0.0,0.0,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17302,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024002,0.0,0.0,0.000000,...,0.000000,0.022648,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
17303,0.011292,0.000000,0.000000,0.000000,0.000000,0.000000,0.011600,0.0,0.0,0.000000,...,0.009815,0.010945,0.000000,0.000000,0.0,0.0,0.0,0.012415,0.013391,0.0
17304,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
17305,0.000000,0.000000,0.000000,0.000000,0.019996,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0


### Nối df vừa sinh với df_train

In [11]:
df_train = pd.concat([df_train, df], axis = 1)

In [12]:
df_train

,score,word_count,essay_length,sentences_count,paragraph_count,clean_text,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,...,tfid_18901,tfid_18902,tfid_18903,tfid_18904,tfid_18905,tfid_18906,tfid_18907,tfid_18908,tfid_18909,tfid_18910
0,3,498,2677,13,1,many people have car where they live. the thin...,56.97,15.1,11.4,7.21,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
1,3,332,1669,19,9,i am a scientist at nasa that is discussing th...,87.35,5.5,6.8,4.52,...,0.014383,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
2,4,550,3077,24,7,people always wish they had the same technolog...,65.05,9.9,11.4,8.36,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
3,4,451,2701,23,9,"we all heard about venus, the planet without a...",58.11,10.5,13.0,10.15,...,0.000000,0.028272,0.015672,0.015793,0.0,0.0,0.0,0.000000,0.000000,0.0
4,3,373,2208,15,11,"dear, state senator this is a letter to argue ...",54.46,11.9,12.7,9.99,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17302,2,157,856,9,5,"the story ""the challenge of exploing venus ""is...",70.94,7.6,10.0,7.60,...,0.000000,0.022648,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
17303,4,579,3354,28,11,technology ha changed a lot of way that we liv...,56.39,11.2,11.7,9.46,...,0.009815,0.010945,0.000000,0.000000,0.0,0.0,0.0,0.012415,0.013391,0.0
17304,2,215,1122,15,5,if you do not like sitting around all day than...,90.39,4.3,6.4,5.33,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
17305,1,231,1427,11,1,"in ""the challenge of exporing venus,"" the auth...",58.42,10.4,11.4,11.20,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0


### Thực hiện truyền dữ liệu qua Pipeline

**Câu hỏi**: Pipeline là gì?

> Pipeline là một cách để mã hóa và tự động hóa các công đoạn và quy trình làm việc cần thiết để tạo ra một mô hình học máy. Pipeline bao gồm nhiều bước tuần tự thực hiện mọi thứ từ trích xuất dữ liệu (data extraction) và tiền xử lý dữ liệu (preprocessing data) cho đến huấn luyện và triển khai mô hình.

> Đối với các hệ thống sử dụng mô hình ML, thì quy trình pipeline là phần trung tâm của sản phẩm. Nó đóng gói toàn bộ các phương pháp xử lý dữ liệu tốt nhất để tạo ra một mô hình học máy phù hợp nhất cho một bộ dữ liệu cụ thể. Ngoài ra pipeline còn cho phép mô hình thực thi trên quy mô lớn. Một thiết kế pipeline end-to-end sẽ cho phép hệ thống của bạn cập nhật một cách thường xuyên các mô hình học máy một cách nhanh chóng.

In [13]:
print("Start running...")
# Define transformers for numerical and categorical columns
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Update categorical and numerical columns
numerical_columns = df_train.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = df_train.select_dtypes('object').columns

# Remove target variable from numerical columns
numerical_columns = numerical_columns.drop('score')

print("- Numerical collumns: ", numerical_columns)
print("\n- Categorical collumns: ", categorical_columns)

# Combine transformers using ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
    ],remainder = 'passthrough')

# Create a pipeline with the preprocessor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)])

Start running...
- Numerical collumns:  Index(['word_count', 'essay_length', 'sentences_count', 'paragraph_count',
       'flesch_reading_ease', 'flesch_kincaid_grade', 'smog_index',
       'coleman_liau_index', 'automated_readability_index',
       'dale_chall_readability_score',
       ...
       'tfid_18901', 'tfid_18902', 'tfid_18903', 'tfid_18904', 'tfid_18905',
       'tfid_18906', 'tfid_18907', 'tfid_18908', 'tfid_18909', 'tfid_18910'],
      dtype='object', length=18924)

- Categorical collumns:  Index(['clean_text'], dtype='object')


In [14]:
# Apply the pipeline to your dataset
X = df_train.drop(['score', 'clean_text'], axis=1)
X = X.apply(pd.to_numeric, errors='coerce')
y = np.log(df_train['score'])

X_preprocessed = pipeline.fit_transform(X)
print("This Pipeline is done")

This Pipeline is done


### X là những cột feature truyền vào Pipeline để chuẩn hóa

In [15]:
X

,word_count,essay_length,sentences_count,paragraph_count,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,automated_readability_index,dale_chall_readability_score,...,tfid_18901,tfid_18902,tfid_18903,tfid_18904,tfid_18905,tfid_18906,tfid_18907,tfid_18908,tfid_18909,tfid_18910
0,498,2677,13,1,56.97,15.1,11.4,7.21,18.0,8.40,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
1,332,1669,19,9,87.35,5.5,6.8,4.52,5.9,6.11,...,0.014383,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
2,550,3077,24,7,65.05,9.9,11.4,8.36,11.1,6.92,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
3,451,2701,23,9,58.11,10.5,13.0,10.15,12.2,7.95,...,0.000000,0.028272,0.015672,0.015793,0.0,0.0,0.0,0.000000,0.000000,0.0
4,373,2208,15,11,54.46,11.9,12.7,9.99,13.5,7.52,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17302,157,856,9,5,70.94,7.6,10.0,7.60,8.0,7.44,...,0.000000,0.022648,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
17303,579,3354,28,11,56.39,11.2,11.7,9.46,12.2,7.21,...,0.009815,0.010945,0.000000,0.000000,0.0,0.0,0.0,0.012415,0.013391,0.0
17304,215,1122,15,5,90.39,4.3,6.4,5.33,5.1,1.52,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
17305,231,1427,11,1,58.42,10.4,11.4,11.20,12.9,8.69,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0


In [16]:
# After Pipeline
# Features columns have been normalized using StandardScaler()
print(X_preprocessed)

[[ 0.86210112  0.65384299 -0.69521865 ... -0.32321807 -0.27946977
  -0.25682696]
 [-0.24169251 -0.43484636 -0.01751654 ... -0.32321807 -0.27946977
  -0.25682696]
 [ 1.2078678   1.08586258  0.54723522 ... -0.32321807 -0.27946977
  -0.25682696]
 ...
 [-1.01966754 -1.02563314 -0.46931795 ... -0.32321807 -0.27946977
  -0.25682696]
 [-0.91327779 -0.69621821 -0.92111936 ... -0.32321807 -0.27946977
  -0.25682696]
 [-1.41862909 -1.38852959 -0.80816901 ... -0.32321807 -0.27946977
  -0.25682696]]


### Chia thành các tập train và tập test cho mô hình

In [17]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, 
                                                    test_size=0.2, random_state=42)

print("Train set shape: ", X_train.shape, " ; ", y_train.shape)
print("Test set shape: ",X_test.shape, " ; " ,y_test.shape)

Train set shape:  (13845, 18924)  ;  (13845,)
Test set shape:  (3462, 18924)  ;  (3462,)


### Cài đặt chấm điểm dựa theo cách chấm điểm của cuộc thi

> Submissions are scored based on the quadratic weighted kappa, which measures the agreement between two outcomes. This metric typically varies from 0 (random agreement) to 1 (complete agreement).

Các bài nộp được tính điểm dựa trên kappa có trọng số bậc hai, thước đo sự phù hợp giữa hai kết quả. Số liệu này thường thay đổi từ 0 (thỏa thuận ngẫu nhiên) đến 1 (thỏa thuận hoàn chỉnh).

In [18]:
# # idea from https://www.kaggle.com/code/rsakata/optimize-qwk-by-lgb/notebook#QWK-objective
def quadratic_weighted_kappa(y_true, y_pred):
    y_pred_exp = np.exp(y_pred)  # Chuyển đổi từ log-space về giá trị thực
    y_pred_exp = y_pred_exp.clip(1, 6).round()  # Giới hạn giá trị giữa 1 và 6 và làm tròn

    y_true_exp = np.round(np.exp(y_true))  # Chuyển đổi từ log-space về giá trị thực
    return cohen_kappa_score(y_true_exp, y_pred_exp, weights='quadratic')

# metric and objective based on public notebooks
def qwk_obj(y_true, y_pred):
    labels = y_true + a
    preds = y_pred + a
    preds = preds.clip(1, 6)
    f = 1/2*np.sum((preds-labels)**2)
    g = 1/2*np.sum((preds-a)**2+b)
    df = preds - labels
    dg = preds - a
    grad = (df/g - f*dg/g**2)*len(labels)
    hess = np.ones(len(labels))
    return grad, hess

a = 2.998
b = 1.092

### Huấn luyện mô hình

## Mô hình XGBoost

In [19]:
# print("Start running...")

# xgb_callbacks = [
#     xgb.callback.EvaluationMonitor(period=25),
#     xgb.callback.EarlyStopping(75, metric_name="QWK", maximize=True, save_best=True)
# ]

# xgb_regressor = xgb.XGBRegressor(
#     objective=qwk_obj,  # Use custom QWK objective function
#     n_estimators= 207,
#     learning_rate= 0.10704240620854825,
#     min_split_loss = 1,
#     max_depth= 6,
#     subsample= 0.6389081081835488,
#     max_bin = 337,
#     random_state=42,
#     num_leaves = 10,
#     extra_trees=True,
#     class_weight='balanced',
#     tree_method="hist"
# )

# # Train the model
# xgb_regressor.fit(X_train, y_train)

## Mô hình LGBM

In [20]:
lgbm_regressor = lgb.LGBMRegressor(
    objective = qwk_obj,
    random_state = 42,
    num_leaves = 9,
    min_data_in_leaf = 85,
    learning_rate = 0.09214428062438888,
    max_depth = 9,
    subsample = 0.6454269174418843,
    n_estimators = 231,
    max_bin = 291,
    bagging_freq = 14)

# Train the model
lgbm_regressor.fit(X_train, y_train)

[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 6.883154 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5398393
[LightGBM] [Info] Number of data points in the train set: 13845, number of used features: 18924
[LightGBM] [Info] Using self-defined objective function


LGBMRegressor(bagging_freq=14, learning_rate=0.09214428062438888, max_bin=291,
              max_depth=9, min_data_in_leaf=85, n_estimators=231, num_leaves=9,
              objective=<function qwk_obj at 0x7c83af4a3910>, random_state=42,
              subsample=0.6454269174418843)

### Thực hiện dự đoán và chấm điểm kết quả

In [21]:
# # Make predictions on the validation set
# y_pred = xgb_regressor.predict(X_test)

# # Calculate the Cohen's kappa score
# score = quadratic_weighted_kappa(y_test, y_pred)
# print(f"Train QWK: {score:.3f}")

In [22]:
# Make predictions on the validation set
y_pred = lgbm_regressor.predict(X_test)

# Calculate the Cohen's kappa score
score = quadratic_weighted_kappa(y_test, y_pred)
print(f"Train QWK: {score:.3f}")

[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
Train QWK: 0.797


### Tinh chỉnh siêu tham số bằng framework Optuna 

### Optuna

> An open source hyperparameter optimization framework to automate hyperparameter search

 **Câu hỏi**: Khó khăn gặp phải khi phải điều chỉnh tham số thủ công?
 
 - Việc chọn siêu tham số khi thực hiện training mô hình dường như đều dựa vào cảm tính và kinh nghiệm của người lập trình. Và điều nay không phải lúc nào cũng có thể làm cho mô hình đạt hiệu suất, kết quả tốt nhất . Còn đối với những lập trình viên chưa có kinh nghiệm lựa chọn siêu tham số thì đây thực sự là một thử thách khi đòi hỏi phải có kiến thức và kinh nghiệm nhất định.

 **Câu hỏi**: Optuna là gì ?
 
 - Optuna là 1 framework hỗ trợ việc tự động điều chỉnh tham số mô hình để mô hình có thể đạt được hiệu năng tốt nhất. Do đó giúp giảm bớt thời gian và công sức khi chúng ta không cần phải điều chỉnh các siêu tham số một cách thủ công nữa. Từ đó giúp tăng năng suất công việc.

#### Định nghĩa objective function

**Trial object definition**: 

> Trial là một đối tượng thể hiện của một class được thực thi trong Optuna.Nó được sử dụng để định nghĩa các siêu tham số được tối ưu. Giá trị được lấy trong phạm vi được định nghĩa, thông tin của các tham số được tìm kiếm trong quá khứ vẫn được giữ lại và giá trị mới sẽ dựa trên những thông tin đó. 

Hay dễ hiểu hơn là Optuna sẽ duyệt qua các tham số được cài đặt thử nghiệm và chọn tập hợp siêu tham số có kết quả tốt nhất là tham số sử dụng cho mô hình cuối cùng.

In [23]:
# def objective(trial):
#     param_grid = {
#         'n_estimators': trial.suggest_int('n_estimators', 500, 700),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
#         'min_split_loss': trial.suggest_int('min_split_loss', 0, 9),
#         'max_depth': trial.suggest_int('max_depth', 6, 12),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'max_bin': trial.suggest_int('max_bin', 256, 512)
#     }

#     xgb_regressor = xgb.XGBRegressor(
#         objective=qwk_obj,
#         random_state=42,
#         tree_method="hist",
#         **param_grid
#     )
    
#     xgb_regressor.fit(X_train, y_train)
    
#     y_pred_log = xgb_regressor.predict(X_test)
#     y_pred = np.exp(y_pred_log)
#     # y_pred = y_pred + a
#     y_pred = y_pred.clip(1, 6).round()
#     y_test_exp = np.exp(y_test)

#     # Calculate the Cohen's kappa score
#     score = cohen_kappa_score(y_test_exp.round(), y_pred, weights='quadratic')
#     return np.mean(score)

In [24]:
# def objective(trial):
#     param_grid = {
#         'num_leaves': trial.suggest_int('num_leaves', 8, 12),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 300),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 60, 120),
#         'max_depth': trial.suggest_int('max_depth', 6, 12),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'max_bin': trial.suggest_int('max_bin', 200, 400),
#         'bagging_freq': trial.suggest_int('bagging_freq', 10, 14)
#     }

#     lgbm_regressor = lgb.LGBMRegressor(
#         objective=qwk_obj,
#         random_state=42,
#         extra_trees = True,
#          class_weight = 'balanced',
#         **param_grid
#     )
    
#     lgbm_regressor.fit(X_train, y_train)
    
#     y_pred_log = lgbm_regressor.predict(X_test)
#     y_pred = np.exp(y_pred_log)
#     # y_pred = y_pred + a
#     y_pred = y_pred.clip(1, 6).round()
#     y_test_exp = np.exp(y_test)

#     # Calculate the Cohen's kappa score
#     score = cohen_kappa_score(y_test_exp.round(), y_pred, weights='quadratic')
#     return np.mean(score)

**Định nghĩa study object**: Để tìm kiếm siêu tham số, bạn cần khởi tạo một đối tượng là study 

> Đối tượng này lưu kết quả tối ưu của bạn.

**Sau đó, sử dụng phương thức optimize**:

- Trong đó, tham số thứ 1 là hàm Objective 
- Tham số thứ 2 là số lượng thử nghiệm. 

> Quá trình tối ưu này được thực hiện trong đối tượng Study và sẽ thực hiện tìm giá trị cực tiểu của các tham số trong hàm Objective bằng phương pháp tối ưu hóa. 

In [25]:
# # Set up the Optuna study
# study = optuna.create_study(direction='maximize', sampler=TPESampler())

# def callback(study, trial):
#     for ii, t in enumerate(study.trials):
#         if t.value == 4:
#             study.stop()
        
# study.optimize(objective, callbacks=[callback])

In [26]:
# # Get the best hyperparameters
# best_params = study.best_params
# print("Best Hyperparameters:", best_params)

### Thực hiện áp dụng các siêu tham số mà Optuna đã thu được và dự đoán kết quả

In [27]:
# # Train the final model with the best hyperparameters
# xgb_regressor = xgb.XGBRegressor(
#     objective=qwk_obj,
#     random_state=42,
#     **best_params
# )

# xgb_regressor.fit(
#     X_train, y_train,
#     early_stopping_rounds=10,
#     eval_set=[(X_test, y_test)],
#     verbose=True
# )

# y_pred_log = xgb_regressor.predict(X_test)
# y_pred = np.exp(y_pred_log)
# # y_pred = y_pred + a
# y_pred = y_pred.clip(1, 6).round()
# y_test_exp = np.exp(y_test)

# # Calculatl the Cohen's kappa score
# score = cohen_kappa_score(y_test_exp.round(), y_pred, weights='quadratic')
# print(f"Train QWK: {score}")

In [28]:
# # Train the final model with the best hyperparameters
# lgbm_regressor = lgb.LGBMRegressor(
#     objective=qwk_obj,
#     random_state=42,
#     **best_params
# )

# lgbm_regressor.fit(
#     X_train, y_train,
#     early_stopping_rounds=10,
#     eval_set=[(X_test, y_test)],
#     verbose=True
# )

# y_pred_log = lgbm_regressor.predict(X_test)
# y_pred = np.exp(y_pred_log)
# # y_pred = y_pred + a
# y_pred = y_pred.clip(1, 6).round()
# y_test_exp = np.exp(y_test)

# # Calculate the Cohen's kappa score
# score = cohen_kappa_score(y_test_exp.round(), y_pred, weights='quadratic')
# print(f"Train QWK: {score}")

### Thực hiện áp dụng Model đã huấn luyện vào file test.csv

In [29]:
df_test

,essay_id,full_text,word_count,essay_length,sentences_count,paragraph_count,clean_text,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,misspelled_count
0,000d118,Many people have car where they live. The thin...,498,2677,13,1,many people have car where they live. the thin...,56.97,15.1,11.4,7.21,18.0,8.40,57.0,13.000000,26
1,000fe60,I am a scientist at NASA that is discussing th...,332,1669,19,9,i am a scientist at nasa that is discussing th...,87.35,5.5,6.8,4.52,5.9,6.11,22.0,6.714286,8
2,001ab80,People always wish they had the same technolog...,550,3077,24,7,people always wish they had the same technolog...,65.05,9.9,11.4,8.36,11.1,6.92,59.0,15.500000,9


In [30]:
# Lưu lại cột essay_id để sử dụng sau này
essay_ids = df_test['essay_id']

test_tfid = vectorizer.transform([i for i in df_test['clean_text']])
dense_matrix = test_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns

df_test = pd.concat([df_test, df], axis = 1)
X_test = df_test.drop(['clean_text', 'essay_id','full_text'], axis=1).astype('float64')

X_test

,word_count,essay_length,sentences_count,paragraph_count,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,automated_readability_index,dale_chall_readability_score,...,tfid_18901,tfid_18902,tfid_18903,tfid_18904,tfid_18905,tfid_18906,tfid_18907,tfid_18908,tfid_18909,tfid_18910
0,498.0,2677.0,13.0,1.0,56.97,15.1,11.4,7.21,18.0,8.40,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,332.0,1669.0,19.0,9.0,87.35,5.5,6.8,4.52,5.9,6.11,...,0.014383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,550.0,3077.0,24.0,7.0,65.05,9.9,11.4,8.36,11.1,6.92,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Xuất kết quả ra file submission và nộp bài

In [31]:
# # Áp dụng pipeline đã huấn luyện để tiền xử lý dữ liệu test
# X_test_preprocessed = pipeline.transform(X_test)

# # Dự đoán điểm số trên dữ liệu test đã tiền xử lý
# y_pred_log = xgb_regressor.predict(X_test_preprocessed)

# # Chuyển đổi ngược từ log-transform
# y_pred = np.exp(y_pred_log)

# # Tạo dataframe chứa essay_id và dự đoán điểm số
# results = pd.DataFrame({'essay_id': essay_ids, 'score': y_pred})

# # Làm tròn điểm số
# results['score'] = round(results['score'])
# results['score'] = results['score'].astype(int)

# # Lưu kết quả ra file csv
# submission = results.to_csv('submission.csv', index=False)

In [32]:
# Áp dụng pipeline đã huấn luyện để tiền xử lý dữ liệu test
X_test_preprocessed = pipeline.transform(X_test)

# Dự đoán điểm số trên dữ liệu test đã tiền xử lý
y_pred_log = lgbm_regressor.predict(X_test_preprocessed)

# Chuyển đổi ngược từ log-transform
y_pred = np.exp(y_pred_log)

# Tạo dataframe chứa essay_id và dự đoán điểm số
results = pd.DataFrame({'essay_id': essay_ids, 'score': y_pred})

# Làm tròn điểm số
results['score'] = round(results['score'])
results['score'] = results['score'].astype(int)

# Lưu kết quả ra file csv
submission = results.to_csv('submission.csv', index=False)

[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14


### In ra kết quả đã dự đoán

In [33]:
results

,essay_id,score
0,000d118,2
1,000fe60,3
2,001ab80,5
